In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1F_zlx6khqP8vvZoBgGru2m-cKoYA4yGY' -O mimic_synthetic_feat.name &> /dev/null
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1O-zoKlFplWdPLV1-OMHWusH-nB9d10jc' -O mimic_synthetic_test.data &> /dev/null
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1SixQE96b8AuAj_nDo-KElSXeo9Vp8BLU' -O mimic_synthetic_train.data &> /dev/null
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&confirm=no_antivirus&id=1LUqLTiYjwsZ3kHfnDABDLZZt65OPax8R' -O mimic_synthetic_train.solution &> /dev/null
print('Data files successfully import')

Data files successfully import


In [ ]:
!pip install pycaret &> /dev/null

In [ ]:
import pandas as pd
from google.colab import files
from pycaret.classification import *

In [ ]:
class FrameWorker:
  def __init__(self, features_names_file):
    columns_names = self.text_to_list(features_names_file)
    self.columns_names = ['index']+columns_names[:len(columns_names)-1]
  
  def text_to_list(self, filename):
    res_list = []
    try:
      with open(filename,'r',encoding='utf-8') as file:
        res_list = file.read().split('\n')
    except Exception as error:
      print(str(error))
    return res_list

  def search_mean_values(self, data):
    data['MEAN_HR_2DAYS'] = data[['HR_MEAN_DAY1','HR_MEAN_DAY2']].mean(axis=1)
    data['MEAN_SYS_BP_2DAYS'] = data[['SYS_BP_MEAN_DAY1','SYS_BP_MEAN_DAY2']].mean(axis=1)
    data['MEAN_DIAS_BP_2DAYS'] = data[['DIAS_BP_MEAN_DAY1','DIAS_BP_MEAN_DAY2']].mean(axis=1)
    data['MEAN_MEAN_BP_2DAYS'] = data[['MEAN_BP_MEAN_DAY1','MEAN_BP_MEAN_DAY2']].mean(axis=1)
    data['MEAN_RESP_RATE_2DAYS'] = data[['RESP_RATE_MEAN_DAY1','RESP_RATE_MEAN_DAY2']].mean(axis=1)
    data['MEAN_TEMP_2DAYS'] = data[['TEMP_MEAN_DAY1','TEMP_MEAN_DAY2']].mean(axis=1)
    data['MEAN_SPO2_2DAYS'] = data[['SPO2_MEAN_DAY1','SPO2_MEAN_DAY2']].mean(axis=1)
    data['MEAN_GLUCOSE_2DAYS'] = data[['GLUCOSE_MEAN_DAY1','GLUCOSE_MEAN_DAY2']].mean(axis=1)
    return data

  def delete_columns(self, data):
    columns_for_deleting = ['ADMITTIME','DISCHTIME','HR_MIN_DAY1', 'HR_MIN_DAY2','HR_MAX_DAY1','HR_MAX_DAY2',
                            'HR_MEAN_DAY1','HR_MEAN_DAY2','SYS_BP_MIN_DAY1','SYS_BP_MIN_DAY2',
                            'SYS_BP_MAX_DAY1','SYS_BP_MAX_DAY2','SYS_BP_MEAN_DAY1','SYS_BP_MEAN_DAY2',
                            'DIAS_BP_MIN_DAY1','DIAS_BP_MIN_DAY2','DIAS_BP_MAX_DAY1','DIAS_BP_MAX_DAY2',
                            'DIAS_BP_MEAN_DAY1','DIAS_BP_MEAN_DAY2','MEAN_BP_MIN_DAY1','MEAN_BP_MIN_DAY2',
                            'MEAN_BP_MAX_DAY1','MEAN_BP_MAX_DAY2','MEAN_BP_MEAN_DAY1','MEAN_BP_MEAN_DAY2',
                            'RESP_RATE_MIN_DAY1','RESP_RATE_MIN_DAY2','RESP_RATE_MAX_DAY1','RESP_RATE_MAX_DAY2',
                            'RESP_RATE_MEAN_DAY1','RESP_RATE_MEAN_DAY2','TEMP_MIN_DAY1','TEMP_MIN_DAY2',
                            'TEMP_MAX_DAY1','TEMP_MAX_DAY2','TEMP_MEAN_DAY1','TEMP_MEAN_DAY2',
                            'SPO2_MIN_DAY1','SPO2_MIN_DAY2','SPO2_MAX_DAY1','SPO2_MAX_DAY2',
                            'SPO2_MEAN_DAY1','SPO2_MEAN_DAY2','GLUCOSE_MIN_DAY1','GLUCOSE_MIN_DAY2',
                            'GLUCOSE_MAX_DAY1','GLUCOSE_MAX_DAY2','GLUCOSE_MEAN_DAY1','GLUCOSE_MEAN_DAY2']
    for column in data.columns:
      if data[column].dtype=='int64' or data[column].dtype=='float64':
        if data[column].mean()==0:
          columns_for_deleting.append(column)
    data = data.drop(columns=columns_for_deleting)
    return data

  def prepare_data(self, data_frame):
    dicts = {} # словарь значений для строковых колонок
    object_columns = [] # колонки со строковыми переменными
    for column_name in data_frame.columns:
      if data_frame[column_name].dtype=='object':
        column_dict = {}
        object_columns.append(column_name)
        elements_list = data_frame[column_name].unique().tolist()
        for index,element in enumerate(elements_list):
          column_dict[element] = index
        dicts[column_name] = column_dict
    for column_name in object_columns:
      data_frame[column_name] = data_frame[column_name].apply(lambda x: int(dicts[column_name][x]))
    return data_frame

  def create_frame(self, frame_file, separator):
    data_frame = pd.read_csv(frame_file,
                             sep=separator,
                             encoding='utf-8',
                             header=None,
                             names=self.columns_names)
    data_frame = self.search_mean_values(data_frame)
    data_frame = self.delete_columns(data_frame)
    data_frame = self.prepare_data(data_frame)
    return data_frame

In [ ]:
fw = FrameWorker('mimic_synthetic_feat.name')

Получение тренировочных данных

In [ ]:
train_df = fw.create_frame('mimic_synthetic_train.data',' ')

In [ ]:
solution = fw.text_to_list('mimic_synthetic_train.solution')[:train_df.shape[0]]

In [ ]:
train_df['solution'] = solution
train_df['solution'] = train_df['solution'].apply(lambda x: '0' if x=='' else x)
train_df['solution'] = train_df['solution'].apply(lambda x: int(x)).astype(int)

In [ ]:
sol_1 = train_df[train_df['solution']==1]
sol_0 = train_df[train_df['solution']==0][len(sol_1):len(sol_1)*25]
res_sol_frame = pd.concat([sol_1, sol_0])

In [ ]:
res_sol_frame.head(2)

Тренировка модели предсказания

In [ ]:
exp1 = setup(res_sol_frame,target='solution')

In [ ]:
best = compare_models(sort = 'AUC')

In [ ]:
model = create_model('ada')

In [ ]:
tuned_model = tune_model(model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.9655,0.7334,0.1333,1.0,0.2353,0.2281,0.3588
1,0.9665,0.7315,0.1590,1.0,0.2743,0.2663,0.3919
2,0.9657,0.7016,0.1385,1.0,0.2432,0.2358,0.3656
3,0.9640,0.6987,0.0974,1.0,0.1776,0.1717,0.3065
4,0.9661,0.7148,0.1531,1.0,0.2655,0.2576,0.3845
5,0.9636,0.7039,0.0918,1.0,0.1682,0.1626,0.2975
6,0.9655,0.7017,0.1378,1.0,0.2422,0.2347,0.3647
7,0.9644,0.7275,0.1077,1.0,0.1944,0.1882,0.3223
8,0.9638,0.7109,0.0923,1.0,0.1690,0.1634,0.2983
9,0.9655,0.7257,0.1333,1.0,0.2353,0.2281,0.3588


In [ ]:
model_final = finalize_model(tuned_model)

Предсказание на тестовых данных

In [ ]:
test_df = fw.create_frame('mimic_synthetic_test.data',' ')

In [ ]:
predictions = predict_model(model_final, data = test_df)

In [ ]:
with open('mimic_synthetic_test.csv','w') as file:
    for label in predictions['Label'].tolist():
        file.write(str(label)+'\n')
files.download("mimic_synthetic_test.csv")        